In [2]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os
username = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
port = 3306

In [3]:
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()


def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

In [4]:
tables = run_sql("show tables;", engine)
for x in tables:
    print(x)

('challengedescriptions',)
('challengewins',)
('chefdetails',)
('episodeinfo',)
('judges',)
('rewards',)


1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series.
The database contains information on both. We are not interested in the ‘Masters’. There
is also a season called “Canada 6” which is part of the Masters series. Create versions of
the chefdetails and judges tables that exclude any records coming from seasons that
include the word Masters or Canada.

In [5]:
chef_no_master_canada = df_select(
    "SELECT * FROM chefdetails where season not like '%Canada%' and season not like '%Master%';",
    engine)
chef_no_master_canada.sample(5)

Retrieved 322 records


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
83,94,Lisa Fernandes,Lisa F.,"Toronto, Ontario",New York,New York,27.0,Chicago,4,US,2,None,None,Female
32,32,Joe Sasto,Joe S.,None,Los Angeles,California,NaN,All-Stars L.A.,17,US,15,None,None,Male
88,99,Andrew D'Ambrosi,Andrew D.,"Fort Lauderdale, Florida",New York,New York,30.0,Chicago,4,US,7,None,None,Female
298,333,Chris Crary,Chris C.,None,Los Angeles,California,29.0,Texas,9,US,9,None,Chef de Cuisine,Male
304,339,Richie Farina,Richie F.,None,Chicago,Illinois,28.0,Texas,9,US,15,None,Sous Chef,Male


In [6]:
judges_no_master_canada = df_select(
    "SELECT * FROM judges where season not like '%Canada%' and season not like '%Master%';",
    engine)
judges_no_master_canada.sample(5)

Retrieved 633 records


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
626,748,World All Stars,20,US,11,Quickfire,Individual,Andrew Wong,None,None
598,696,Texas,9,US,15,Elimination,Individual,Cammi Granato,None,None
336,349,Kentucky,16,US,14,Elimination,Individual,Florita Morais Alves,None,None
267,280,Houston,19,US,4,Quickfire,Individual,Chris Williams,None,None
144,157,Charleston,14,US,9,Elimination,Individual,Michael Voltaggio,Yes,"Tournament of Champions, Battle of the Brothers"


2. Further trim your chefdetails and judges tables by including only the columns of
interest from the table descriptions above.

In [7]:
filtered_chef_trimmed = df_select(
    "SELECT season, seasonNumber, name, placement, gender FROM chefdetails where season not like '%Canada%' and season not like '%Master%';",
    engine)
filtered_chef_trimmed.sample(5)

Retrieved 322 records


,season,seasonNumber,name,placement,gender
266,San Francisco,1,Andrea Beaman,7,Female
110,D.C.,7,Arnold Myint,13,Male
222,New Orleans,11,Michael Sichel,14,Male
31,All-Stars L.A.,17,Angelo Sosa,14,Male
162,Las Vegas,6,Michael Voltaggio,1,Male


In [8]:
filtered_judges_trimmed = df_select(
    "SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC FROM judges where season not like '%Canada%' and season not like '%Master%';",
    engine)
filtered_judges_trimmed.sample(5)

Retrieved 633 records


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
333,Kentucky,16,13,Elimination,Jowett Yu,None
559,Seattle,10,7,Quickfire,Stephanie Izard,Yes
224,D.C.,7,12,Elimination,Eric Ripert,None
30,All Stars: New York,8,14,Quickfire,Lorena Garcia,None
419,Miami,3,3,Elimination,Norman van Aken,None


3. Your tables should have information from 20 remaining different seasons, numbered
from 1 to 20. Make a table that has two columns, one called season and one called
seasonNumber showing the number associated with each season. (For example, San
Francisco is season 1.)


In [10]:
with engine.connect() as conn:
    season_and_number = pd.read_sql(
        sqla.text("""SELECT distinct judges.season, judges.seasonNumber
        FROM judges join chefdetails on judges.season = chefdetails.season 
        where judges.season not like '%Master%' and judges.season not like '%Canada%' order by seasonNumber;"""), conn
    )
season_and_number

,season,seasonNumber
0,San Francisco,1
1,Los Angeles,2
2,Miami,3
3,Chicago,4
4,New York,5
5,Las Vegas,6
6,D.C.,7
7,All Stars: New York,8
8,Texas,9
9,Seattle,10


4. Answer the following questions using your tables so far.
a. Among all chef contestants, how many are male and how many are female?

In [9]:
filtered_chef_trimmed['gender'].value_counts()

Male      170
Female    152
Name: gender, dtype: int64

b. Among all winners (placement==1) how many are male and how many are female?

In [10]:
fct_win = filtered_chef_trimmed[filtered_chef_trimmed['placement'] == 1]
fct_win['gender'].value_counts()

Male      14
Female     6
Name: gender, dtype: int64

There are 6 female winners and 14 male winners.

c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many
are female?

In [39]:
fct_female = filtered_chef_trimmed[filtered_chef_trimmed['gender'] == 'Female']
one = fct_female[fct_female['placement'] == 1].count()
two = fct_female[fct_female['placement'] == 2].count()
three = fct_female[fct_female['placement'] == 3].count()
female_total = one+two+three
str(female_total['placement']) +' female'

'25 female'

In [42]:
fct_male = filtered_chef_trimmed[filtered_chef_trimmed['gender'] == 'Male']
one_male = fct_male[fct_male['placement'] == 1].count()
two_male = fct_male[fct_male['placement'] == 2].count()
three_male = fct_male[fct_male['placement'] == 3].count()
male_total = one_male+two_male+three_male
str(male_total['placement']) + ' male'

'36 male'

There are 25 female values in the top 3 and 36 males in the top 3.

d. Compare the number of times a female was placed in the top 3 with the number of times
a female won. Does this number seem unusual? That is, are women who place in the
top 3 less likely than expected to ultimately win the competition? Why or why not?

In [49]:
female_total['placement'] / (female_total['placement']+male_total['placement']) *100

40.98360655737705

In [50]:
6 / (6+14) * 100

30.0

In [51]:
6/female_total['placement'] *100

24.0

In [52]:
14/male_total['placement'] * 100

38.88888888888889

Out of everyone in the top 3, women make up 40.98% of the top 3. Out of those, only 6 or 24% of have won TopChef. Since there are 6 women out of the 20 winners, this means that women have won 30% of the time. When we compare those number to the amount of time mean have won with the amount of men in the top 3, this puts women at as less likely to win competition. The amount of men in the top 3 is 60% this is actually a rather large comparison, however when you look at the proportion of women and men in the top three who have won, they come out to roughly the same proportion of ~40%. So while the otherall number of women in the top 3 is low, the number of female winners in proportion to the number of females contestants in the top 3.

5. Make a table with one row for each first place winner containing the winner’s name,
the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they
served as a guest judge or not.

In [59]:
chef_guest_judge = filtered_judges_trimmed[filtered_judges_trimmed['competedOnTC'] == 'Yes']
chef_judge_list = []
for i in range(1, len(chef_guest_judge['guestJudge'])):
    if chef_guest_judge.iloc[i]['guestJudge'] not in chef_judge_list:
        chef_judge_list.append(chef_guest_judge.iloc[i]['guestJudge'])

In [60]:
chef_info = df_select("""SELECT season, seasonNumber, name
    FROM (select placement, name, season, seasonNumber from chefdetails where placement = 1) as subquery
    where season not like '%Canada%' and season not like '%Master%';""", engine)

Retrieved 20 records


In [61]:
chef_info['guest_judge?'] = chef_info['name'].apply(lambda x: 'Yes' if x in chef_judge_list else 'No')
chef_info

,season,seasonNumber,name,guest_judge?
0,All Stars: New York,8,Richard Blais,Yes
1,All-Stars L.A.,17,Melissa King,Yes
2,Boston,12,Mei Lin,No
3,California,13,Jeremy Ford,No
4,Charleston,14,Brooke Williamson,Yes
5,Chicago,4,Stephanie Izard,Yes
6,D.C.,7,Kevin Sbraga,No
7,Colorado,15,Joseph Flamm,No
8,Houston,19,Kah-wai Lo,No
9,Kentucky,16,Kelsey Barnard Clark,Yes


6. Find all contestants who were not first place winners but did serve as a guest judge at
some point. Make sure to only list each name once.


In [85]:
chef_not_win = df_select("""SELECT subquery.season, subquery.seasonNumber, subquery.name
    FROM (select distinct name, placement, season, seasonNumber from chefdetails where placement != 1) as subquery
    inner join judges on subquery.name = judges.guestJudge
    where subquery.season not like '%Canada%' and subquery.season not like '%Master%'group by subquery.name;""", engine)
chef_not_win

Retrieved 32 records


,season,seasonNumber,name
0,Seattle,10,Brooke Williamson
1,Chicago,4,Richard Blais
2,Chicago,4,Antonia Lofaso
3,New Orleans,11,Carlos Gaytan
4,World All Stars,20,Nicole Gomes
5,Los Angeles,2,Sam Talbot
6,Las Vegas,6,Bryan Voltaggio
7,Las Vegas,6,Mike Isabella
8,Chicago,4,Spike Mendelsohn
9,World All Stars,20,Dawn Burrell
